In [3]:
import pandas as pd
import re

import bootcampviztools2 as bt

In [2]:
# Para que funcione necesitas bajarte los archivos de datos de Kaggle 
df = pd.read_csv("./data/train.csv", index_col = ["laptop_ID"])
df =df[["Cpu","Price_in_euros"]]

FileNotFoundError: [Errno 2] No such file or directory: './data/train.csv'

In [30]:
df

,Cpu,Price_in_euros
laptop_ID,,
755,Intel Core i3 6006U 2GHz,539.00
618,Intel Core i7 6700HQ 2.6GHz,879.01
909,Intel Core i7 7500U 2.7GHz,900.00
2,Intel Core i5 1.8GHz,898.94
286,Intel Core i3 6006U 2.0GHz,428.00
...,...,...
28,Intel Core i5 8250U 1.6GHz,800.00
1160,Intel Core i5 6300U 2.4GHz,1629.00
78,Intel Core i5 7200U 2.5GHz,519.00


In [18]:
import pandas as pd
import numpy as np
import re

def parse_memory(memory_str):
    # Diccionario base
    components = {
        "primary_capacity_gb": np.nan,
        "primary_type": np.nan,
        "has_secondary": 0,
        "secondary_capacity_gb": np.nan,
        "secondary_type": np.nan,
        "total_capacity_gb": np.nan
    }
    
    # Expresión regular para extraer capacidad y tipo
    pattern = r"(\d+\.?\d*)(TB|GB|MB)\s*(\w+\s*\w*)"
    
    # Separar combinaciones (ej: "256GB SSD + 1TB HDD")
    drives = re.split(r"\s*\+\s*", memory_str)
    
    # Procesar cada disco
    capacities = []
    types = []
    
    for drive in drives:
        match = re.search(pattern, drive)
        if match:
            value, unit, drive_type = match.groups()
            value = float(value)
            
            # Convertir todo a GB
            multiplier = 1000 if unit == "TB" else 1 if unit == "GB" else 0.001
            capacity_gb = value * multiplier
            
            capacities.append(capacity_gb)
            types.append(drive_type.strip())
    
    if capacities:
        # Primario
        components["primary_capacity_gb"] = capacities[0]
        components["primary_type"] = types[0]
        
        # Secundario (si existe)
        if len(capacities) > 1:
            components["has_secondary"] = 1
            components["secondary_capacity_gb"] = capacities[1]
            components["secondary_type"] = types[1]
        
        # Capacidad total
        components["total_capacity_gb"] = sum(capacities)
    
    return components

In [19]:
# Cargar datos
df = pd.read_csv("prueba_Memory.csv")

# Aplicar la función
parsed_memory = df["Memory"].apply(
    lambda x: pd.Series(parse_memory(x))
)

# Combinar con el DataFrame original
df = pd.concat([df, parsed_memory], axis=1)

In [20]:
print(df["primary_type"].value_counts())

primary_type
SSD              587
HDD              256
Flash Storage     63
Hybrid             6
Name: count, dtype: int64


In [21]:
print(df.groupby(pd.cut(df["total_capacity_gb"], bins=5))["Price_in_euros"].mean())

total_capacity_gb
(5.496, 508.8]       955.920059
(508.8, 1009.6]     1163.540350
(1009.6, 1510.4]    1524.064628
(1510.4, 2011.2]    1748.336316
(2011.2, 2512.0]    1504.082222
Name: Price_in_euros, dtype: float64


C:\Users\Viajes Elkano\AppData\Local\Temp\ipykernel_95108\501770051.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  print(df.groupby(pd.cut(df["total_capacity_gb"], bins=5))["Price_in_euros"].mean())


In [22]:
df

,Memory,Price_in_euros,primary_capacity_gb,primary_type,has_secondary,secondary_capacity_gb,secondary_type,total_capacity_gb
0,256GB SSD,539.00,256.0,SSD,0,NaN,NaN,256.0
1,1TB HDD,879.01,1000.0,HDD,0,NaN,NaN,1000.0
2,1TB HDD,900.00,1000.0,HDD,0,NaN,NaN,1000.0
3,128GB Flash Storage,898.94,128.0,Flash Storage,0,NaN,NaN,128.0
4,1TB HDD,428.00,1000.0,HDD,0,NaN,NaN,1000.0
...,...,...,...,...,...,...,...,...
907,256GB SSD,800.00,256.0,SSD,0,NaN,NaN,256.0
908,256GB SSD,1629.00,256.0,SSD,0,NaN,NaN,256.0
909,2TB HDD,519.00,2000.0,HDD,0,NaN,NaN,2000.0
910,500GB HDD,258.00,500.0,HDD,0,NaN,NaN,500.0
